# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [38]:

!pip install tensorflow
!pip install pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

df_data = pd.read_csv('data/flavia/all.csv')

print(df_data.head())

train_df, temp_df = train_test_split(df_data, test_size=0.4, random_state=42, stratify=df_data['y'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['y'])

# Kiểm tra kích thước các tập dữ liệu
print(f'Số lượng ảnh trong tập train: {len(train_df)}')
print(f'Số lượng ảnh trong tập validation: {len(val_df)}')
print(f'Số lượng ảnh trong tập test: {len(test_df)}')

img_height, img_width = 244, 244
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2,
    zoom_range=0.2,  
    horizontal_flip=True,
    fill_mode='nearest'
)


val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_df['y'] = train_df['y'].astype(str)
val_df['y'] = val_df['y'].astype(str)
test_df['y'] = test_df['y'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='data/flavia', 
    x_col='id',
    y_col='y',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='data/flavia',
    x_col='id',
    y_col='y',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='data/flavia',
    x_col='id',
    y_col='y',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=50,
                    callbacks=[early_stopping, reduce_lr])


test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')


python(80713) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(80716) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   Unnamed: 0        id   y
0           0  1300.jpg   5
1           1  3152.jpg  23
2           2  1439.jpg   9
3           3  1243.jpg   4
4           4  1186.jpg   3
Số lượng ảnh trong tập train: 1144
Số lượng ảnh trong tập validation: 381
Số lượng ảnh trong tập test: 382
Found 1144 validated image filenames belonging to 32 classes.
Found 381 validated image filenames belonging to 32 classes.
Found 382 validated image filenames belonging to 32 classes.


ImportError: cannot import name 'MobileNetV3' from 'tensorflow.keras.applications' (/Users/hotien/miniconda3/lib/python3.12/site-packages/keras/_tf_keras/keras/applications/__init__.py)

In [42]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')

/Users/hotien/miniconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 373ms/step - accuracy: 0.9530 - loss: 0.4999
Test Accuracy: 0.96


In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Lưu mô hình TFLite
with open('leaf_recognition_model_flavia.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/yp/zzs1vcsn3_j60vflhd6qr_m40000gn/T/tmpk9n4ps4x/assets


INFO:tensorflow:Assets written to: /var/folders/yp/zzs1vcsn3_j60vflhd6qr_m40000gn/T/tmpk9n4ps4x/assets


Saved artifact at '/var/folders/yp/zzs1vcsn3_j60vflhd6qr_m40000gn/T/tmpk9n4ps4x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 244, 244, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)
Captures:
  5994024976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994026896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994026320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994026512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994027280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994027472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994025360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994026704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994025168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994027856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5994028816: Ten

W0000 00:00:1735397345.486312 21799270 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1735397345.486889 21799270 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-28 21:49:05.506612: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/yp/zzs1vcsn3_j60vflhd6qr_m40000gn/T/tmpk9n4ps4x
2024-12-28 21:49:05.512418: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-28 21:49:05.512426: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/yp/zzs1vcsn3_j60vflhd6qr_m40000gn/T/tmpk9n4ps4x
I0000 00:00:1735397345.576049 21799270 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-12-28 21:49:05.585380: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-28 21:49:05.893842: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/fol

In [47]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(244, 244))  # Thay đổi kích thước ảnh
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Thêm một chiều cho batch
    img_array /= 255.0  # Chuẩn hóa pixel về khoảng [0, 1]
    return img_array

# Đường dẫn đến ảnh
image_path = 'test_image/orange.jpg'  # Thay thế bằng đường dẫn đến ảnh của bạn

# Tiền xử lý ảnh
image = preprocess_image(image_path)

# Nạp mô hình TFLite
interpreter = tf.lite.Interpreter(model_path='leaf_recognition_model_flavia.tflite')  # Thay thế bằng đường dẫn đến mô hình của bạn
interpreter.allocate_tensors()

# Lấy thông tin input và output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Đưa dữ liệu vào mô hình
interpreter.set_tensor(input_details[0]['index'], image)

# Chạy suy luận
interpreter.invoke()

# Lấy kết quả đầu ra
output_data = interpreter.get_tensor(output_details[0]['index'])

# Xử lý kết quả đầu ra
predicted_class_index = np.argmax(output_data)

# Lấy danh sách nhãn từ train_generator
class_labels = list(train_generator.class_indices.keys())

# Hiển thị kết quả
predicted_class_label = class_labels[predicted_class_index]

# Đọc file Excel để lấy nhãn
labels_file = 'labels.xlsx'  # Đường dẫn đến file Excel
df_labels = pd.read_excel(labels_file, header=None)  # Đọc file Excel, không có header
# Giả định cột A1 là số thứ tự, A2 là tên khoa học, A3 là tên phổ thông
df_labels.columns = ['Index', 'Scientific Name', 'Common Name']

# Lấy nhãn tương ứng
predicted_label = df_labels[df_labels['Index'] == int(predicted_class_label)]
scientific_name = predicted_label['Scientific Name'].values[0]
common_name = predicted_label['Common Name'].values[0]

# Hiển thị kết quả
print(f'Ảnh được dự đoán thuộc lớp: {common_name} ({scientific_name})')

Ảnh được dự đoán thuộc lớp: tangerine (Citrus reticulata Blanco)


In [16]:
class_labels

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']